## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [11]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1=tf.Variable(shape=[28*28, 100],
                            dtype=tf.float32,
                            initial_value=tf.random.uniform(shape=[28*28, 100],minval=-0.1,maxval=0.1))
        self.b1=tf.Variable(shape=[100],dtype=tf.float32,initial_value=tf.zeros(100))
        self.W2=tf.Variable(shape=[100, 20],
                            dtype=tf.float32,
                            initial_value=tf.random.uniform(shape=[100, 20],minval=-0.1,maxval=0.1))
        self.b2=tf.Variable(shape=[20],dtype=tf.float32,initial_value=tf.zeros(20))
        self.train_variables = [self.W1, self.W2, self.b1, self.b2]
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [12]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [13]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 3.0105784 ; accuracy 0.07428333
epoch 1 : loss 2.9981136 ; accuracy 0.08293334
epoch 2 : loss 2.9858 ; accuracy 0.09076667
epoch 3 : loss 2.973633 ; accuracy 0.09893333
epoch 4 : loss 2.9616084 ; accuracy 0.10763333
epoch 5 : loss 2.9497223 ; accuracy 0.11593334
epoch 6 : loss 2.937971 ; accuracy 0.124283336
epoch 7 : loss 2.9263508 ; accuracy 0.13148333
epoch 8 : loss 2.9148588 ; accuracy 0.13873333
epoch 9 : loss 2.9034913 ; accuracy 0.14678334
epoch 10 : loss 2.8922448 ; accuracy 0.15471667
epoch 11 : loss 2.8811169 ; accuracy 0.16288333
epoch 12 : loss 2.8701048 ; accuracy 0.17075
epoch 13 : loss 2.8592052 ; accuracy 0.17876667
epoch 14 : loss 2.848416 ; accuracy 0.18578333
epoch 15 : loss 2.837735 ; accuracy 0.1936
epoch 16 : loss 2.827159 ; accuracy 0.20091666
epoch 17 : loss 2.816686 ; accuracy 0.20758334
epoch 18 : loss 2.806314 ; accuracy 0.2147
epoch 19 : loss 2.7960413 ; accuracy 0.22196667
epoch 20 : loss 2.785865 ; accuracy 0.22841667
epoch 21 : loss 2.77578